<a href="https://colab.research.google.com/github/zh19980811/Code_Perference_Benchmark/blob/%E5%91%BD%E5%90%8D%E8%AF%84%E4%BC%B0/python_%E5%91%BD%E5%90%8D_%E8%AF%84%E4%BC%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import ast
import re
import keyword
import json
from collections import defaultdict, Counter
import matplotlib.pyplot as plt

# --- 命名风格分类器 ---
def classify_naming_style(name):
    if name.startswith('__') and name.endswith('__'):
        return 'dunder_method'
    if name.startswith('_') and not name.startswith('__'):
        return 'private'
    if re.fullmatch(r'[a-z]+(_[a-z0-9]+)*', name):
        return 'snake_case'
    if re.fullmatch(r'[A-Z][a-zA-Z0-9]+', name):
        return 'PascalCase'
    if re.fullmatch(r'[a-z]+([A-Z][a-z0-9]*)+', name):
        return 'camelCase'
    if re.fullmatch(r'[A-Z]+(_[A-Z0-9]+)*', name):
        return 'UPPER_CASE'
    return 'invalid'

# --- 风格统计分析（字符级） ---
def compute_style_stats(names):
    total_chars = sum(len(name) for name in names)
    stats = {'uppercase': 0, 'lowercase': 0, 'underscore': 0, 'digit': 0, 'symbol': 0}
    for name in names:
        for char in name:
            if char.isupper():
                stats['uppercase'] += 1
            elif char.islower():
                stats['lowercase'] += 1
            elif char == '_':
                stats['underscore'] += 1
            elif char.isdigit():
                stats['digit'] += 1
            else:
                stats['symbol'] += 1
    return {k + '_ratio': round(v / total_chars, 3) if total_chars else 0 for k, v in stats.items()}

# --- 风格分布统计 ---
def compute_style_distribution(names):
    styles = [classify_naming_style(name) for name in names]
    counter = Counter(styles)
    return dict(counter)

# --- 命名提取器 ---
class NameExtractor(ast.NodeVisitor):
    def __init__(self):
        self.names = defaultdict(set)

    def visit_FunctionDef(self, node):
        self.names['function'].add(node.name)
        for arg in node.args.args:
            self.names['parameter'].add(arg.arg)
        self.generic_visit(node)

    def visit_ClassDef(self, node):
        self.names['class'].add(node.name)
        self.generic_visit(node)

    def visit_Name(self, node):
        self.names['variable'].add(node.id)
        self.generic_visit(node)

    def visit_Assign(self, node):
        for target in node.targets:
            if isinstance(target, ast.Name):
                name = target.id
                if name.isupper():
                    self.names['constant'].add(name)
                else:
                    self.names['variable'].add(name)
        self.generic_visit(node)

    def visit_Attribute(self, node):
        if isinstance(node.value, ast.Name) and node.value.id == 'self':
            self.names['attribute'].add(node.attr)
        self.generic_visit(node)

def extract_names(code: str):
    tree = ast.parse(code)
    extractor = NameExtractor()
    extractor.visit(tree)
    return extractor.names

# --- 命名规范评分 ---
def check_naming_rules(name: str, kind: str) -> dict:
    result = {}
    score = 0
    total = 0

    total += 1
    result['starts_with_letter'] = name[0].isalpha()
    score += int(result['starts_with_letter'])

    if kind == 'class':
        total += 1
        result['is_pascal_case'] = bool(re.match(r'^[A-Z][a-zA-Z0-9]+$', name))
        score += int(result['is_pascal_case'])

    elif kind in ['function', 'variable', 'parameter', 'attribute']:
        total += 1
        result['is_snake_case'] = bool(re.match(r'^[a-z]+(_[a-z0-9]+)*$', name))
        score += int(result['is_snake_case'])

    elif kind == 'constant':
        total += 1
        result['is_upper_case'] = bool(re.match(r'^[A-Z]+(_[A-Z0-9]+)*$', name))
        score += int(result['is_upper_case'])

    elif kind == 'magic_method':
        total += 1
        result['is_dunder'] = name.startswith('__') and name.endswith('__')
        score += int(result['is_dunder'])

    elif kind == 'private':
        total += 1
        result['starts_with_underscore'] = name.startswith('_') and not name.startswith('__')
        score += int(result['starts_with_underscore'])

    total += 1
    result['is_keyword'] = name in keyword.kwlist
    score += int(not result['is_keyword'])

    result['score'] = round(score / total, 2)
    return result

# --- 分布图绘制 ---
def plot_style_distribution(dist1, dist2):
    all_styles = sorted(set(dist1) | set(dist2))
    vals1 = [dist1.get(style, 0) for style in all_styles]
    vals2 = [dist2.get(style, 0) for style in all_styles]

    x = range(len(all_styles))
    plt.figure(figsize=(10, 6))
    plt.bar([i - 0.2 for i in x], vals1, width=0.4, label='Code1')
    plt.bar([i + 0.2 for i in x], vals2, width=0.4, label='Code2')
    plt.xticks(x, all_styles, rotation=45)
    plt.ylabel('Count')
    plt.title('Naming Style Distribution')
    plt.legend()
    plt.tight_layout()
    plt.savefig("style_distribution_comparison.png")
    plt.close()

# --- 整合主函数 ---
def analyze_and_compare(code1: str, code2: str, json_path="naming_full_analysis.json"):
    names1 = extract_names(code1)
    names2 = extract_names(code2)

    def score_all(names):
        scores = {}
        for kind, name_list in names.items():
            for name in name_list:
                key = f"{kind}:{name}"
                scores[key] = check_naming_rules(name, kind)
        return scores

    scores1 = score_all(names1)
    scores2 = score_all(names2)

    avg1 = round(sum(v['score'] for v in scores1.values()) / len(scores1), 2) if scores1 else 0
    avg2 = round(sum(v['score'] for v in scores2.values()) / len(scores2), 2) if scores2 else 0

    improved = []
    regressed = []

    for name in scores2:
        if name in scores1:
            delta = scores2[name]['score'] - scores1[name]['score']
            if delta > 0:
                improved.append(name)
            elif delta < 0:
                regressed.append(name)

    all_names1 = [name for names in names1.values() for name in names]
    all_names2 = [name for names in names2.values() for name in names]

    result = {
        "summary": {
            "code1_avg_score": avg1,
            "code2_avg_score": avg2,
            "improved_names": improved,
            "regressed_names": regressed
        },
        "code1_scores": scores1,
        "code2_scores": scores2,
        "style_stats": {
            "code1": compute_style_stats(all_names1),
            "code2": compute_style_stats(all_names2)
        },
        "style_distribution": {
            "code1": compute_style_distribution(all_names1),
            "code2": compute_style_distribution(all_names2)
        }
    }

    plot_style_distribution(result["style_distribution"]["code1"], result["style_distribution"]["code2"])

    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(result, f, indent=4, ensure_ascii=False)

    print(f"✅ 命名规范和风格分析已保存至 {json_path}")
    return result

# --- 示例代码 ---
if __name__ == "__main__":
    code1 = '''
PI = 3.14
class my_class:
    def processData(self):
        pass
    def __init__(self, x, y):
        self.Value = x + y
        return self.Value
'''

    code2 = '''
MAX_RETRIES = 5
class MyClass:
    def process_data(self):
        pass
    def __init__(self, x, y):
        self.value = x + y
        return self.value
'''

    analyze_and_compare(code1, code2)


✅ 命名规范和风格分析已保存至 naming_full_analysis.json


明白了！你是想要一份**文字版的功能说明文档**，可以直接用在论文或 GitHub 项目中，例如放在 `README.md` 或论文中的方法章节里。下面是我为你整理的一份完整文档说明👇

---

# 📘 Python 命名规范与风格分析系统说明文档

## 🧩 系统简介

本项目实现了一个用于分析和对比两个 Python 代码片段命名规范质量的自动化系统。系统结合静态代码分析（AST）与规则引擎，从**可读性、风格一致性、命名规范性**等多个角度出发，对 Python 命名进行质量评分和风格分类。

该系统广泛适用于：

- Python 项目命名风格审核
- 教学/培训中代码规范对比
- 学术论文中的实验对照
- GitHub 项目自动命名检测与展示

---

## 🔧 系统功能模块

### 1. 命名元素提取（AST 分析）

使用 Python 内置的 `ast` 模块，将代码解析为抽象语法树，从中提取以下命名元素：

- 类名（`class`）
- 函数名（`def`）
- 函数参数名
- 局部/全局变量名
- 常量名（全大写）
- 类属性名（如 `self.name`）

---

### 2. 命名规范评分

对每个命名元素基于其类型（类、函数、变量等）应用命名规则进行打分：

| 命名类型 | 检查规则                   |
|----------|----------------------------|
| 类名     | 是否符合 PascalCase 风格   |
| 函数/变量 | 是否符合 snake_case 风格   |
| 常量     | 是否为 UPPER_CASE          |
| 私有变量 | 是否以单下划线开头         |
| 魔术方法 | 是否为 `__xxx__` 格式     |
| 全部类型 | 是否以字母开头、是否为保留字等 |

每条规则通过得 1 分，总分计算平均，输出 `score ∈ [0, 1]`。

---

### 3. 命名质量对比分析

比较两段代码中所有命名的得分，输出：

- 两段代码平均得分（整体规范程度）
- 每个命名是否**改进**（得分提升）或**退步**
- 输出详细对比分数 `code1_scores` 与 `code2_scores`

---

### 4. 命名风格分类器

为每个命名自动识别其命名风格，包括：

- `snake_case`
- `PascalCase`
- `camelCase`
- `UPPER_CASE`
- `dunder_method`（如 `__init__`）
- `private`（以下划线开头）
- `invalid`（不符合任何已知风格）

可用于统计命名风格分布、分析风格混乱情况。

---

### 5. 字符风格统计分析

统计所有命名中字符级别的使用情况：

| 统计项         | 含义                       |
|----------------|----------------------------|
| uppercase_ratio | 大写字母占比               |
| lowercase_ratio | 小写字母占比               |
| underscore_ratio | 下划线 `_` 占比           |
| digit_ratio     | 数字占比                   |
| symbol_ratio    | 其他符号占比（应接近 0）  |

帮助判断风格倾向（如 PascalCase vs snake_case）。

---

### 6. 可视化输出（风格分布图）

自动生成柱状图，展示两段代码中命名风格的分布情况，保存为图片 `style_distribution_comparison.png`，可用于展示或报告插图。

---

### 7. JSON 输出结构说明

输出主文件为：`naming_full_analysis.json`，包含字段如下：

```json
{
  "summary": {
    "code1_avg_score": 0.76,
    "code2_avg_score": 0.91,
    "improved_names": [...],
    "regressed_names": [...]
  },
  "code1_scores": { ... },
  "code2_scores": { ... },
  "style_stats": { ... },
  "style_distribution": { ... }
}
```

---

## 🚀 使用方式（示例）

```python
from analysis_module import analyze_and_compare

code1 = '''class my_class: def doThing(): pass'''
code2 = '''class MyClass: def do_thing(): pass'''

analyze_and_compare(code1, code2)
```

将输出：

- 分析 JSON 报告：`naming_full_analysis.json`
- 风格分布对比图：`style_distribution_comparison.png`

---

## 📚 适用场景举例

- ✅ 学术论文中对比重构前后的命名风格变化
- ✅ 教学案例中学生代码命名规范评分
- ✅ 企业项目中命名审查和规范制定依据
- ✅ 自动化代码评审工具插件组件

---

如果你希望我再生成对应的 `README.md` 或 `.tex` 格式的论文模块内容，我也可以直接导出！要不要我转成 Markdown 文件？📄